# Assigment 5

**Group members:** <br>
ZUÑIGA ROMERO, ADRIANA VIOLETA <br>
PLASENCIA CUSTODIO, CLAUDIA JIMENA <br>
ECHEVARRIA CARHUANCHO, FRANCISCO ANDRES <br>
HUAMAN GARCIA, YANELL VALERYN <br>
VERA ROMAN, JOSSELYNE ANTUANE 

### Solution

### 5.1 Instalación e importación de Librerías:

In [ ]:
!pip install selenium 
# this library is to manipulate browser:
!pip install webdriver-manager
!pip install html5lib

In [ ]:
#To control the layout and width within Notebook:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [ ]:
# Import necessary modules from Selenium:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service #To configure the WebDriver service:
from selenium.webdriver.common.by import By #Provides methods for locating elements:
from selenium.webdriver.support.ui import Select #for interacting with dropdowns
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd 
import numpy as np
import os
import time
import re
import unidecode
import time 

### 5.2  Initialize the Chrome driver

#### Function to select a specific type of electoral process

Using a function to set up the Chrome WebDriver provides a modular and reusable way to configure and initialize your WebDriver instances

In [ ]:
def setup_driver():
    #First, set the path to the ChromeDriver executable
    service = Service(executable_path="chromedriver-win64/chromedriver.exe")
    # Configure options for the Chrome browser
    options = webdriver.ChromeOptions()
    # Initialize the Chrome WebDriver with the specified service and options
    driver = webdriver.Chrome(service=service, options=options)
    # Maximize the browser window
    driver.maximize_window()
    # Return the configured WebDriver instance
    return driver

### 5.3 Navigating the page: Election

#### Function to navigate to a given URL

This function provides a clean interface for the main script or other functions to use.

In [ ]:
def go_to_url(driver, url):
    # Navigate the WebDriver to the specified URL
    driver.get(url)

### 5.4 Construction of the loop and export of information in an Excel file"Elecciones_1931_to_2021"

#### Function to select a specific type of electoral process

This function utilizes the Selenium WebDriver to select a specific process type in a web interface. It begins by clicking on an element to open a dropdown, followed by a short pause for the dropdown to load. Then, it clicks on another element to choose the desired process type, with an additional pause to confirm the selection. While these pauses can be necessary for interface responsiveness, excessive use may affect code efficiency.

In [ ]:
def select_process_type(driver, tipo_proceso_xpath, tipo_xpath):
    # Click on the element specified by tipo_proceso_xpath to open the process type dropdown
    driver.find_element(By.XPATH, tipo_proceso_xpath).click()
    
    # Pause for 2 seconds to allow the dropdown to appear
    time.sleep(2)
    
    # Click on the element specified by tipo_xpath to select the desired process type
    driver.find_element(By.XPATH, tipo_xpath).click()
    # Pause for 2 seconds to ensure the selection is registered
    time.sleep(2)

#### Function to choose the presidential electoral process

This function selects a presidential process type on a web interface. With two parameters, driver representing the browser instance and eleccion_tipo_xpath denoting the XPath of the element triggering the selection, the function clicks on the specified element to initiate the selection of the presidential process type. A 2-second pause (time.sleep(2)) follows to ensure proper interface loading.

In [ ]:
def choose_presidential_process(driver, eleccion_tipo_xpath):
    # Click on the element specified by eleccion_tipo_xpath to choose the presidential process
    driver.find_element(By.XPATH, eleccion_tipo_xpath).click()
    # Pause for 2 seconds to allow for the selection to be processed
    time.sleep(2)

#### Function to get the options for the electoral process

This function aims to interact with a dropdown specified by the XPath elecciones_xpath and return the available options within that dropdown. It utilizes the Select class from Selenium, which provides convenient methods for handling dropdowns. The options method is used here to retrieve the list of options within the specified dropdown.

In [ ]:
def get_process_options(driver, elecciones_xpath):
    # Find the dropdown element using the specified XPath, then create a Select object to interact with the dropdown and finally return the options of the dropdown
    return Select(driver.find_element(By.XPATH, elecciones_xpath)).options